## Process all the data using TPP

In [33]:
# TODO: Delete this cell before release
# #Change the names of all the fastq files to SRA numbers for testing
# from shutil import copyfile
# import pandas as pd

# samp_df = pd.read_csv('input/Sample_Labels.csv')
# for uid, sra in zip(samp_df['Run_UID'],samp_df['SRA Accession']):
#     f1 = 'input/'+uid + '_1.fastq'
#     f1_out = 'input/'+sra + '_1.fastq'
#     f2 = 'input/'+uid + '_2.fastq'
#     f2_out = 'input/'+sra + '_2.fastq'
    
#     copyfile(f1, f1_out, follow_symlinks=False)
#     copyfile(f2, f2_out, follow_symlinks=False)

In [ ]:
%%bash
#SET THESE VARIABLES FOR YOUR LOCAL FILE STRUCTURE:
PATH_TO_DATA=input #Note: Any fastq files located here will be processed for alignment with tpp
OUT_DIR=output
REFS=input/Genome/MAC109.fa
GENBANK_FILE=input/Genome/MAC109.gb

#Definition of variables for processing
PYTHON2=$(which python2)
BWA=$(which bwa)
BWA_ALG="aln"

REPLICON_ID="CP029332,CP029333,CP029334"
FASTQ_DIR=$PATH_TO_DATA

PREFIXES_OUTFILE=$OUT_DIR/`basename $FASTQ_DIR`_prefixes.txt

# These are used for creating a CSV file
CSV_OUTFILE=$OUT_DIR/`basename $FASTQ_DIR`.csv
UNIQUE_FIELDS="locus_tag"
FIELDS="product regulatory_class bound_moiety"

#Parameter settings for tpp
PRIMER=AACCTGTTA
MISMATCHES=2
WINDOW_SIZE=6

###################################################################

#Process raw fastq files using tpp
COUNTER=0
INITIAL_START_TIME=$SECONDS
for FASTQ in $FASTQ_DIR/*_1.fastq; do
#for FASTQ in $FASTQ_DIR/*53606_1.fastq; do
    echo "******** Run $COUNTER: $FASTQ ********"
    READS1=$FASTQ
    READS2=${FASTQ/_1.fastq/_2.fastq}

    OUTNAME=$(basename $FASTQ)
    OUTNAME=${OUTNAME/_1.fastq/}
    tpp -himar1 -bwa $BWA -bwa-alg $BWA_ALG -ref $REFS -replicon-ids $REPLICON_ID -reads1 $READS1 -reads2 $READS2 -window-size $WINDOW_SIZE -primer $PRIMER -mismatches $MISMATCHES -output $OUT_DIR/$OUTNAME &
done
wait

echo "Creating prefixes file with all prefixes from all runs..."
basename -a $OUT_DIR/*.wig | cut -c-10 | uniq > $PREFIXES_OUTFILE
echo "Created '$PREFIXES_OUTFILE'."
echo ""
echo "Creating CSV file with all samples processed by TPP..."
$PYTHON2 scripts/wig_gb_to_csv.py -l $PREFIXES_OUTFILE -g $GENBANK_FILE -u $UNIQUE_FIELDS -f $FIELDS -o $CSV_OUTFILE
echo "Created '$CSV_OUTFILE'."
echo ""
(( TOTAL_RUN_TIME = SECONDS - INITIAL_START_TIME )) 
echo "********** TPP driver script finished in a total of $TOTAL_RUN_TIME seconds **********"